In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
# MOvie DB https://grouplens.org/datasets/movielens/
# https://files.grouplens.org/datasets/movielens/ml-32m.zip 
# https://files.grouplens.org/datasets/movielens/ml-32m-README.html

# Projektas (kažkiek dienų trukmės)
# (Čia ne ML, kurkite savo algoritmą, arba savo algoritmą + ML):
# Filmų rekomendacijos skriptas (Jupyter notebook)  arba Streamlit aplikacija
# Rekomenduoti filmus pagal pasirinkimus (remiantis kitų vartotojų vertinimais) - vartotojas įveda žanrą, aktorių, etc .. 
# (apribokite įvestį 3-5 parametrais)
# Rezultatas - rekomenduojami filmai ( 1, 2 ar daugiau)
# get_dummies() -> dummy_df = df["Fruits"].str.get_dummies(", ") 
# iš čia: https://stackoverflow.com/questions/76783764/pandas-get-dummies-on-rows-with-multiple-entries

In [3]:
df_movies = pd.read_csv(r"C:\Users\pauli\Downloads\ml-32m\movies.csv")
df_ratings = pd.read_csv(r"C:\Users\pauli\Downloads\ml-32m\ratings.csv")
df_tags = pd.read_csv(r"C:\Users\pauli\Downloads\ml-32m\tags.csv")

In [4]:
df_movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [5]:
df_ratings.head(3)

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976


In [6]:
df_tags.head(3)

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469


In [7]:
print(len(df_movies))
print(len(df_ratings))
print(len(df_tags))

87585
32000204
2000072


In [28]:
df_ratings1 = df_ratings.groupby('movieId').agg(
    mean_rating=('rating', 'mean'),
    rating_count=('rating', 'count')
)

In [8]:
# df_ratings1 = df_ratings.groupby('movieId')['rating'].mean()

In [29]:
df_ratings1.head()

,mean_rating,rating_count
movieId,,
1,3.897438,68997
2,3.275758,28904
3,3.139447,13134
4,2.845331,2806
5,3.059602,13154


In [30]:
df_bendras = pd.merge(df_movies, df_ratings1, on='movieId', how='left')

In [31]:
df_bendras.head(3)

,movieId,title,genres,mean_rating,rating_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.897438,68997.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.275758,28904.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.139447,13134.0


In [12]:
df_tags1 = df_tags.groupby(['movieId'])['tag'].apply(lambda x: '|'.join(map(str, x))).reset_index()
df_tags1['tag'] = df_tags1['tag'].str.lower()

In [13]:
df_bendras = pd.merge(df_bendras, df_tags1, on='movieId', how='left')

In [14]:
dftest = df_bendras[df_bendras['genres'].str.contains('Action', na=False)]

In [15]:
dftest.head()

,movieId,title,genres,rating,tag
5,6,Heat (1995),Action|Crime|Thriller,3.868277,atmospheric|dialogue|tumey's dvds|atmospheric|...
8,9,Sudden Death (1995),Action,2.987723,action|jean-claude van damme|can't remember|ac...
9,10,GoldenEye (1995),Action|Adventure|Thriller,3.427850,james bond|tumey's dvds|007|bond|casual sex|es...
14,15,Cutthroat Island (1995),Action|Adventure|Romance,2.730881,adventure|dumb but funny|matthew modine|pirate...
19,20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller,2.871458,afternoon section|lame|worthwhile|action hero|...


In [16]:
dftest = dftest.sort_values('rating', ascending=False)

In [17]:
dftest = dftest[dftest['rating'].between(1, 2, inclusive='both')]

In [ ]:
# bandome Mykolo variant1.

In [72]:
dfM = pd.merge(df_ratings, df_movies, on='movieId', how='left')

In [35]:
dfM.head()

,userId,movieId,rating,timestamp,title,genres
8,1,110,3.0,943231119,Braveheart (1995),Action|Drama|War
15,1,260,5.0,943228696,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
23,1,541,5.0,943227521,Blade Runner (1982),Action|Sci-Fi|Thriller
31,1,908,5.0,944250228,North by Northwest (1959),Action|Adventure|Mystery|Romance|Thriller
53,1,1196,5.0,943228696,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi


In [73]:
dfM = dfM[dfM['genres'].str.contains('Action', na=False)]

In [74]:
dfM2 = dfM.groupby('userId')['genres'].count().reset_index()
dfM2.rename(columns={'genres': 'balsiukai'}, inplace=True)

In [75]:
dfM = pd.merge(dfM, dfM2, on='userId', how='left')

In [76]:
dfM  = dfM [dfM ['balsiukai'] >= 20]

In [90]:
dfMr = dfM.groupby('movieId')['rating'].mean().reset_index()
dfMr =  pd.merge(dfMr, dfM, on='movieId', how='left')

In [91]:
dfMr = dfMr.sort_values('rating_x', ascending=False).reset_index()
dfMr = dfMr.drop(['movieId','userId','rating_y'], axis=1)

# st.write(dftest.head(1000))

In [92]:
dfMr.head()

,index,rating_x,timestamp,title,genres,balsiukai
0,8631531,5.0,1581355059,The Sword Stained with Royal Blood (1993),Action|Romance,795
1,8717859,5.0,1695212087,Decision (2011),Action|Children|Thriller,52
2,8717858,5.0,1695189581,Champions (2008),Action|Drama,52
3,8312751,5.0,1558737997,Tarzan's Hidden Jungle (1955),Action|Adventure|Drama|Romance,886
4,8488446,5.0,1558738192,Crosshairs (2013),Action|Drama|Thriller,886
